### Ejercicio 5 :
- [x] Integrar librerías necesarias
- [x] Definición de un esquema
- [x] Crear DataFrame de Esquema
- [x] Cambiar nombre de columnas 
- [x] Mostrar DataFrame
- [x] Eliminar columnas 
- [x] Mostrar DataFrame
- [x] Filtrar valores nulos
- [x] Reemplazar valores numéricos nulos a 0
- [x] Eliminar valores nulos

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType
from pyspark.sql.functions import col, when

In [2]:
schema = StructType([
    StructField("Name", StringType(), True),
    StructField("Age", IntegerType(), True),
    StructField("Gender", StringType(), True)
])

In [3]:
data = [
    ("David", 19, "M"),
    ("Mark", 25, "M"),
    ("Shirley", None, "F"),
    ("Jasmine", 50, None),
    ("John", None, None)
]

In [4]:
spark = SparkSession.builder \
    .appName("Exercise5") \
    .getOrCreate()

24/07/26 12:54:37 WARN Utils: Your hostname, Macbook-Air-31.local resolves to a loopback address: 127.0.0.1; using 192.168.68.102 instead (on interface en0)
24/07/26 12:54:37 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/07/26 12:54:37 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
df = spark.createDataFrame(data, schema)

In [6]:
df.show()

+-------+----+------+
|   Name| Age|Gender|
+-------+----+------+
|  David|  19|     M|
|   Mark|  25|     M|
|Shirley|NULL|     F|
|Jasmine|  50|  NULL|
|   John|NULL|  NULL|
+-------+----+------+



In [7]:
df_renamed = df.withColumnRenamed("Name", "FullName").withColumnRenamed("Age", "Years")
df_renamed.show()

+--------+-----+------+
|FullName|Years|Gender|
+--------+-----+------+
|   David|   19|     M|
|    Mark|   25|     M|
| Shirley| NULL|     F|
| Jasmine|   50|  NULL|
|    John| NULL|  NULL|
+--------+-----+------+



In [8]:
df_dropped = df_renamed.drop("Gender")
df_dropped.show()

+--------+-----+
|FullName|Years|
+--------+-----+
|   David|   19|
|    Mark|   25|
| Shirley| NULL|
| Jasmine|   50|
|    John| NULL|
+--------+-----+



In [9]:
df_filtered = df_renamed.filter(df_renamed.Years.isNotNull())
df_filtered.show()

+--------+-----+------+
|FullName|Years|Gender|
+--------+-----+------+
|   David|   19|     M|
|    Mark|   25|     M|
| Jasmine|   50|  NULL|
+--------+-----+------+



In [10]:
df_filled = df_renamed.withColumn("Years", when(col("Years").isNull(), 0).otherwise(col("Years")))
df_filled.show()

+--------+-----+------+
|FullName|Years|Gender|
+--------+-----+------+
|   David|   19|     M|
|    Mark|   25|     M|
| Shirley|    0|     F|
| Jasmine|   50|  NULL|
|    John|    0|  NULL|
+--------+-----+------+



In [11]:
df_no_nulls = df_renamed.na.drop()
df_no_nulls.show()

+--------+-----+------+
|FullName|Years|Gender|
+--------+-----+------+
|   David|   19|     M|
|    Mark|   25|     M|
+--------+-----+------+



In [12]:
df_filled.write.csv("ej5_csv", header=True)
df_filled.show()

+--------+-----+------+
|FullName|Years|Gender|
+--------+-----+------+
|   David|   19|     M|
|    Mark|   25|     M|
| Shirley|    0|     F|
| Jasmine|   50|  NULL|
|    John|    0|  NULL|
+--------+-----+------+



In [13]:
spark.stop()